In [1]:
import sys
sys.path.insert(0, '/tf/utils/')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pesq import pesq
import pystoi
from scipy.signal import lfilter, wiener
import scipy.linalg 
from tqdm import tqdm

from utils import load_wav, calculate_snr, itakura_distortion, performance, preemphasis
from sound import Sound
from batch import DataGenerator

from multiprocessing import Pool, cpu_count
import librosa

In [3]:
base_shape_size = 8000

In [4]:
# carrega sons de ruido e sons de voz
sound_base = Sound('../Dados/Base/', '../Dados/ESC-50-master/audio/', base_shape_size)

Loading noise files: 100%|██████████| 2000/2000 [00:09<00:00, 219.85it/s]


In [5]:
# Parâmetros do sistema
order = 16  # Ordem da análise LPC
window_size = 250  # Tamanho da janela
sample_rate = 8000  # Taxa de amostragem (exemplo)

In [6]:
data_generator = DataGenerator(sound_base.train_X, sound_base.noise_sounds)

In [7]:
def preemphasis_filter(signal, preemphasis_coeff):
    # Aplicar filtro de pré-ênfase de alta passagem
    preemphasized_signal = np.append(signal[0], signal[1:] - preemphasis_coeff * signal[:-1])
    return preemphasized_signal

In [26]:
def lpc_analysis(signal, order):
    # windowed_signal = preemphasis_filter(signal, 0.83)
    windowed_signal = signal * np.hanning(len(signal))

    a = librosa.lpc(windowed_signal, order=order)

    b = np.hstack([[0], -1 * a[1:]])
    y_hat = scipy.signal.lfilter(b, [1], signal)

    var_residual = np.var(signal - y_hat)

    return a, np.sqrt(var_residual), 0

In [27]:
def build_matrices(A, window_size):
    Ak = np.zeros((order, order))
    Ak[:, 0] = -A[1:]
    Ak[:-1, 1:] = np.eye(order - 1)

    H = np.zeros((1, order))
    H[0, 0] = 1.0

    return Ak, H

In [28]:
def kalman_filter(signal, Ak, H, Q, R):
    x_hat = np.zeros(order)  # Estado estimado
    P = np.eye(order)  # Covariância estimada

    filtered_signal = []

    for sample in signal:
        # Atualização temporal (Predição)
        x_hat = np.dot(Ak, x_hat)
        P = np.dot(np.dot(Ak, P), Ak.T) + Q

        # Atualização de mensuração (Correção)
        K = np.dot(np.dot(P, H.T), np.linalg.inv(np.dot(np.dot(H, P), H.T) + R))
        x_hat = x_hat + np.dot(K, (sample - np.dot(H, x_hat)))
        P = P - np.dot(np.dot(K, H), P)

        filtered_signal.append(x_hat[0])  # Apenas a primeira componente é o sinal estimado

    return np.array(filtered_signal)

In [29]:
def process_signal(signal, window_size, order, sample_rate, SNR_dB=10.):
    filtered_signal = []

    for i in range(0, len(signal), window_size):
        window_samples = signal[i:i+window_size]
        
        # Realizar análise LPC e construir as matrizes Ak e H
        A, sigma, _ = lpc_analysis(window_samples, order)
        Ak, H = build_matrices(A, len(window_samples))

        # Calcular a variância do erro de aquisição R com base no SNR linear
        SNR_linear = 10.**(SNR_dB / 10.)
        Rx = 1. / SNR_linear
        
        # Calcular Q e R (assumindo que não mudam dentro da janela)
        Q = np.eye(order) * sigma  # Variância do erro de predição
        R = np.eye(1) * Rx  # Variância do erro de aquisição

        # Aplicar o filtro de Kalman na janela
        filtered_window = kalman_filter(window_samples, Ak, H, Q, R)
        filtered_signal.extend(filtered_window)

    return np.array(filtered_signal)

In [30]:
def process_batch(args):
    x_batch, y_batch, SNR_dB_batch, window_size, order, sample_rate = args
    filtered_batch = [process_signal(noisy_signal, window_size, order, sample_rate, SNR_dB=SNR_dB) for noisy_signal, SNR_dB in zip(x_batch, SNR_dB_batch)]
    
    pesq_scores = [pesq(8000, clean, filtered.reshape(-1), 'nb') for clean, filtered in zip(y_batch, filtered_batch)]
    stoi_scores = [pystoi.stoi(clean, filtered, 8000) for clean, filtered in zip(y_batch, filtered_batch)]
    snr_scores = [calculate_snr(clean, filtered) for clean, filtered in zip(y_batch, filtered_batch)]
    ID_scores = [itakura_distortion(clean, filtered, window_size, order) for clean, filtered in zip(y_batch, filtered_batch)]
    
    return pesq_scores, stoi_scores, snr_scores, ID_scores

In [31]:
batch_num = 50
df_resultado = pd.DataFrame()

In [ ]:
num_processes = cpu_count()  # Usar o número de núcleos da CPU
    
with Pool(processes=num_processes) as pool:
    results = []
    
    for _ in tqdm(range(batch_num)):
        x_batch, y_batch, metrics_batch_df = next(data_generator.generate_sample_metrics(window_size, order, batch_size=128))
        SNR_dB_batch = metrics_batch_df['SNR']
        
        args = (x_batch, y_batch, SNR_dB_batch, window_size, order, sample_rate)
        result = pool.apply_async(process_batch, (args,))
        results.append((result, metrics_batch_df))
    
    df_resultado = pd.DataFrame()
    
    for result, metrics_batch_df in results:
        pesq_scores, stoi_scores, snr_scores, ID_scores = result.get()
        metrics_batch_df['PESQ (Filtered)'] = pesq_scores
        metrics_batch_df['STOI (Filtered)'] = stoi_scores
        metrics_batch_df['SNR (Filtered)'] = snr_scores
        metrics_batch_df['ID (Filtered)'] = ID_scores
        
        df_resultado = pd.concat([df_resultado, metrics_batch_df], ignore_index=True)

  2%|▏         | 1/50 [00:07<06:27,  7.90s/it]Process ForkPoolWorker-18:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Process ForkPoolWorker-16:
Traceback (most recent call last):
Process ForkPoolWorker-15:
Process ForkPoolWorker-20:
Traceback (most recent call last):
  File "/tmp/ipykernel_122661/2873436523.py", line 3, in process_batch
    filtered_batch = [process_signal(noisy_signal, window_size, order, sample_rate, SNR_dB=SNR_dB) for noisy_signal, SNR_dB in zip(x_batch, SNR_dB_batch)]
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/pytho

In [ ]:
# for _ in tqdm(range(batch_num)):
#     x_batch, y_batch, metrics_batch_df = next(data_generator.generate_sample_metrics(window_size, order, batch_size=128))
                
#     # Aplica o filtro de Kalman aos sinais ruidosos
#     filtered_batch = [process_signal(noisy_signal, window_size, order, sample_rate, SNR_dB=SNR_dB) for noisy_signal, SNR_dB in zip(x_batch, metrics_batch_df['SNR'])]
    
#     # Calcula as métricas para os sinais filtrados
#     pesq_scores = [pesq(8000, clean, filtered.reshape(-1), 'nb') for clean, filtered in zip(y_batch, filtered_batch)]
#     stoi_scores = [pystoi.stoi(clean, filtered, 8000) for clean, filtered in zip(y_batch, filtered_batch)]
#     snr_scores = [calculate_snr(clean, filtered) for clean, filtered in zip(y_batch, filtered_batch)]
#     ID_scores = [itakura_distortion(clean, filtered, window_size, order) for clean, filtered in zip(y_batch, filtered_batch)]
    
#     # Adiciona as métricas dos sinais filtrados ao DataFrame
#     metrics_batch_df['PESQ (Filtered)'] = pesq_scores
#     metrics_batch_df['STOI (Filtered)'] = stoi_scores
#     metrics_batch_df['SNR (Filtered)'] = snr_scores
#     metrics_batch_df['ID (Filtered)'] = ID_scores

#     df_resultado = pd.concat([df_resultado, metrics_batch_df], ignore_index=True)

In [ ]:
df_resultado

In [ ]:
df_resultado.describe()

In [ ]:
# index = 6
# start = 2500
# end = start + 750

# # Plote o sinal original e o sinal do LPC gerado
# plt.figure(figsize=(10, 6))

# # Plote o sinal original
# plt.subplot(3, 1, 1)
# plt.plot(y_batch[index][start:end], label='Sinal Original', color='blue')
# plt.title('Sinal Original')
# plt.xlabel('Amostras')
# plt.ylabel('Amplitude')
# plt.legend()

# # Plote o sinal do LPC gerado
# plt.subplot(3, 1, 2)
# plt.plot(x_batch[index][start:end], label='Sinal Ruidoso', color='red')
# plt.title('Sinal Ruidoso')
# plt.xlabel('Amostras')
# plt.ylabel('Amplitude')
# plt.legend()

# # Plote o sinal do LPC gerado
# plt.subplot(3, 1, 3)
# plt.plot(filtered_batch[index][start:end], label='Sinal Filtrado', color='green')
# plt.title('Sinal Filtrado')
# plt.xlabel('Amostras')
# plt.ylabel('Amplitude')
# plt.legend()

# plt.tight_layout()
# plt.show()

In [ ]:
performance(df_resultado, 'Kalman')

In [23]:
from datetime import datetime

In [24]:
# Get the current datetime
current_datetime = datetime.now()

# Format the datetime as a string to use in the file name
datetime_str = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")

# Define the file name with the datetime
file_name = f"kalman_{datetime_str}.xlsx"

In [25]:
df_resultado.to_excel(file_name, index=False)